## 4.1.6 재현 신경망(Recurrent Neural Network) 분류 모델

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import json

## 학습 데이터 파일 로드

In [3]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

TRAIN_INPUT_DATA = 'train_input.npy'
TRAIN_LABEL_DATA = 'train_label.npy'
DATA_CONFIGS = 'data_configs.json'

In [14]:
input_data = np.load(open(DATA_IN_PATH + TRAIN_INPUT_DATA, 'rb'))
label_data = np.load(open(DATA_IN_PATH + TRAIN_LABEL_DATA, 'rb'))
prepro_configs = None

with open(DATA_IN_PATH + DATA_CONFIGS, 'r') as f:
    prepro_configs =json.load(f)

In [48]:
TEST_SPLIT = 0.1
RANDOM_SEED = 13371447

input_train, input_eval, label_train, label_eval = train_test_split(input_data, label_data, test_size = TEST_SPLIT, random_state=RANDOM_SEED)

In [49]:
import tensorflow as tf

BATCH_SIZE = 16
NUM_EPOCHS = 3

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((input_train, label_train))
    dataset = dataset.shuffle(buffer_size=50000)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.repeat(count=NUM_EPOCHS)
    return dataset

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((input_eval, label_eval))
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

# 데이터셋 정의
train_dataset = train_input_fn()
eval_dataset = eval_input_fn()


## 모델 정의

In [50]:
# 하이퍼파라미터 값 정의
VOCAB_SIZE = prepro_configs['vocab_size']+1
WORD_EMBEDDING_DIM = 100
HIDDEN_STATE_DIM = 150
DENSE_FEATURE_DIM = 150

learning_rate = 0.001

In [51]:
print(len(prepro_configs['vocab']), VOCAB_SIZE)

74065 74066


In [52]:
# 모델 정의 함수
def create_model(vocab_size, embedding_dim, hidden_dim, dense_dim):
    # Sequential 모델을 정의합니다.
    model = tf.keras.Sequential([
        # Embedding 층: 단어를 고정된 크기의 벡터로 변환합니다.
        # vocab_size: 어휘 사전의 크기
        # embedding_dim: 임베딩 벡터의 차원
        # input_length: 입력 시퀀스의 길이 (None으로 설정 시 가변 길이 입력 허용)
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=None),

        # Dropout 층: 과적합을 방지하기 위해 20%의 뉴런을 무작위로 비활성화합니다.
        tf.keras.layers.Dropout(0.2),

        # LSTM 층: 첫 번째 LSTM 층으로, return_sequences=True로 설정되어 모든 타임스텝의 출력을 반환합니다.
        # hidden_dim: LSTM의 출력 차원
        tf.keras.layers.LSTM(hidden_dim, return_sequences=True),

        # LSTM 층: 두 번째 LSTM 층으로, 마지막 타임스텝의 출력만 반환합니다.
        tf.keras.layers.LSTM(hidden_dim),

        # Dropout 층: 과적합을 방지하기 위해 다시 20%의 뉴런을 무작위로 비활성화합니다.
        tf.keras.layers.Dropout(0.2),

        # Dense 층: 완전 연결층으로, tanh 활성화 함수를 사용하여 dense_dim 차원의 출력을 생성합니다.
        tf.keras.layers.Dense(dense_dim, activation='tanh'),

        # Dropout 층: 과적합을 방지하기 위해 또다시 20%의 뉴런을 무작위로 비활성화합니다.
        tf.keras.layers.Dropout(0.2),

        # Dense 층: 최종 출력층으로, sigmoid 활성화 함수를 사용하여 이진 분류 결과를 출력합니다.
        # 1개의 유닛을 가지며, 0~1 사이의 값을 출력합니다.
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    # 모델 컴파일: 옵티마이저, 손실 함수, 평가 지표 설정
    # Adam 옵티마이저 사용, 손실 함수로 이진 교차 엔트로피 사용, 정확도를 평가 지표로 설정
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    # 모델 반환
    return model


In [53]:

# 데이터 경로가 존재하지 않으면 생성
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

# 모델 생성
model = create_model(VOCAB_SIZE, WORD_EMBEDDING_DIM, HIDDEN_STATE_DIM, DENSE_FEATURE_DIM)

# 모델 구조 확인
model.summary()

# 모델 저장
model.save(os.path.join(DATA_OUT_PATH, 'rnn_model.keras'))

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_api.py:100: UserWarning: You are saving a model that has not yet been built. It might not contain any weights yet. Consider building the model first by calling it on some data.
  saving_lib.save_model(model, filepath)


In [22]:
# tensorflow 1.x 에서 사용

# def model_fn(features, labels, mode):
#     TRAIN = mode == tf.estimator.ModeKeys.TRAIN
#     EVAL = mode == tf.estimator.Modelkeys.EVAL
#     PREDICT = mode == tf.estimator.Modelkeys.PREDICT

#     # feature의 경우 딕셔너리 형태로 구성되어 있기 때문.
#     # tf.keras.layers.Embedding 함수가 해당 임베딩 층(Embedding Layer)를 거치도록 도와주는 함수이다.
#     # 임베딩 층의 경우 단어의 수치적 표현, 차원축소, 유사성 학습을 진행할 수 있게 도와주는 층이라고 보면 된다.
#     embedding_layer = tf.keras.layers.Embedding(VOCAB_SIZE, WORD_EMBEDDING_DIM)(features['x'])
#     embedding_layer = tf.keras.layers.Dropout(0.2)(embedding_layer)

#     # 순환 신경망을 사용하기 위한 함수
#     # 해당 객체는 하나의 LSTM Cell 을 의미하며 해당 Cell 객체를 여러개 생성해서 하나의 리스트로 만들어 준다.
#     rnn_layers = [tf.nn.rnn_cell.LSTMCell(size) for size in [HIDDEN_STATE_DIM, HIDDEN_STATE_DIM]]
#     # LTSMCell을 쌓게 되면 하나의 MultiRNN을 묶어야, 즉 래핑(wrapping)을 해야한다. 해당 multiRNNCell을 생성함으로써 스택 구조의 LSTM 신경망을 구현할 수 있다.
#     multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)

#     # 네트워크와 임베딩 벡터와 연산하기 위해 dynamic_rnn 함수를 선언한다.
#     # 텐서플로의 dynamic_rnn: for 구문 활용없이 자동으로 순환신경망을 만들어 주는 역할을 한다
#     outputs, state = tf.nn.dynamic_rnn(cell = multi_rnn_cell,
#                                        inputs=embedding_layer,
#                                        dtype = tf.float32)

#     outputs = tf.keras.layers.Dropout(0.2)(outputs)
#     hidden_layer = tf.keras.layers.Dense(DENSE_FEATURE_DIM, activation=tf.nn.tanh)(outpus[:, -1, :])
#     hidden_layer = tf.keras.layers.Dropout(0.2)(hidden_layer)
#     # 최종적으로 1dim의 output만 할 수 있도록 dense layer 를 활용해 차원 변환을 한다
#     logits = tf.keras.layers.Dense(1)(hidden_layer)
#     logits = tf.squeeze(logits, axis = -1)

#     loss = tf.losses.sigmoid_cross_entropy(labels, logits)

#     # 훈련 데이터에 대하여 경사하강법을 진행하는 부분
#     if TRAIN:
#         global_step = tf.train.get_global_step()
#         # 경사하강법 중 하나인 아담 옵티마이저를 활용하여 경사하강을 진행
#         train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step)

#         return tf.estimator.EstimatorSpec(mode=mode, train_op=train_op, loss=loss)

#     # 모델 출력값에 대하여 0~ 1 사이의 값으로 정의하기 위해 sigmoid 함수를 사용
#     sigmoid_logits = tf.nn.sigmoid(logits)

#     # 모델 평가에 대한 코드
#     if EVAL:
#         accuracy = tf.metrics.accuracy(laels, tf.round(sigmoid_logits))
#         eval_metric_ops = {'acc':accuracy}

#         return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=eval_metric_ops)
        
#     # 모델 예측을 위한 코드
#     if PREDICT:
#         # 모델 출력값에 대하여 0~ 1 사이의 값으로 정의하기 위해 sigmoid 함수를 사용
#         predictions = {'sentiment': sigmoid_logits}

#         return tf.estimator.EstimatorSpec(mode = mode, predictions = predictions)

In [43]:
# if not os.path.exists(DATA_OUT_PATH):
#     os.makedirs(DATA_OUT_PATH)

# est = tf.estimator.Estimator(model_fn=model_fn,
#                              model_dir=DATA_OUT_PATH + 'checkpoint/rnn') 

In [54]:
os.environ["CUDA_VISIBLE_DEVICES"]="4"


# # 모델 훈련
# model.fit(input_train, label_train, epochs=10, validation_data=(input_eval, label_eval))


# 모델 훈련
model.fit(train_dataset, epochs=NUM_EPOCHS, validation_data=eval_dataset)

# tensorflow 1.x 에서 사용
# est.train(train_input_fn)

Epoch 1/3
4221/4221 ━━━━━━━━━━━━━━━━━━━━ 760s 180ms/step - accuracy: 0.5772 - loss: 0.6356 - val_accuracy: 0.8720 - val_loss: 0.3344
Epoch 2/3
4221/4221 ━━━━━━━━━━━━━━━━━━━━ 762s 180ms/step - accuracy: 0.9627 - loss: 0.1119 - val_accuracy: 0.8572 - val_loss: 0.5427
Epoch 3/3
4221/4221 ━━━━━━━━━━━━━━━━━━━━ 774s 183ms/step - accuracy: 0.9949 - loss: 0.0244 - val_accuracy: 0.8556 - val_loss: 0.8312


In [ ]:
# est.evaluate(eval_input_fn)

## 캐글 평가 데이터셋 만들기

In [55]:
DATA_OUT_PATH = './data_out/'
TEST_INPUT_DATA = 'test_input.npy'
TEST_ID_DATA = 'test_id.npy'

test_input_data = np.load(open(DATA_IN_PATH + TEST_INPUT_DATA, 'rb'))

In [59]:
# test_dataset = tf.data.Dataset.from_tensor_slices({'x': test_input_data})
# test_dataset = test_dataset.batch(BATCH_SIZE)

In [67]:
import tensorflow as tf
test_dataset = tf.data.Dataset.from_tensor_slices(test_input_data)
test_dataset = test_dataset.batch(BATCH_SIZE)

# 예측 수행
predictions = model.predict(test_dataset)

# predictions는 이제 예측 결과를 포함합니다.


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 78s 50ms/step


In [62]:
# predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":test_input_data}, shuffle=False)

In [71]:
# predictions = np.array([p['sentiment'] for p in est.predict(input_fn=predict_input_fn)])

In [75]:
# TensorFlow 2.x 코드
predictions = np.array(predictions)
predictions


array([[9.9961269e-01],
       [3.9450437e-04],
       [2.1139032e-03],
       ...,
       [4.4497382e-04],
       [9.9961323e-01],
       [5.8831203e-01]], dtype=float32)

In [79]:
test_id = np.load(open(DATA_IN_PATH + TEST_ID_DATA, 'rb', ), allow_pickle=True)

In [81]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

output = pd.DataFrame(data={"id": list(test_id), "sentiment":list(predictions)} )
output.to_csv(DATA_OUT_PATH + 'movie_review_result_rnn.csv', index=False, quoting=3 )